In [ ]:
import re
import time
from pathlib import Path

from selenium.webdriver import Edge, EdgeOptions
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait


def fun_页面标题等待(title: str, driver: Edge):
    return WebDriverWait(driver=driver, timeout=10, poll_frequency=1).until(
        lambda d: expected_conditions.title_contains(title)
    )


def fun_js点击(selector: str, driver: Edge):
    driver.execute_script(f'document.querySelector("{selector}").click()')


def fun_获取数字(stem: str) -> int:
    num_list = re.findall(r"\d+", stem)
    if len(num_list) > 0:
        return int("".join(num_list))
    return 0


def fun_获取ele数字(ele: WebElement) -> int:
    text = ele.find_element(By.CSS_SELECTOR, ".name").text
    num = re.findall(r"xq_(\d+)", text)
    num = int("".join(num))
    return num


options = EdgeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--single-process")

driver = Edge(options=options)
driver.implicitly_wait(20)
driver.set_window_rect(x=1274, y=0, width=1292, height=1398)

driver.get(
    "https://havanalogin.taobao.com/mini_login.htm?lang=zh_CN&appName=taobao&appEntrance=qianniu_pc_web&styleType=vertical&bizParams=&notLoadSsoView=true&notKeepLogin=false&isMobile=false&cssUrl=https://g.alicdn.com/qn/qn-login-iframe-css/0.0.8/qn-login-iframe.css&returnUrl=https://myseller.taobao.com/home.htm&rnd=0.17371788128615084"
)
fun_页面标题等待("千牛商家工作台", driver)

In [ ]:
start_path = Path(r"D:\泡泡素材\0000-0999\0733")
start_stem = fun_获取数字(start_path.stem)

# 构建所有需要处理的文件夹
all_path = []
for material_path in start_path.parent.iterdir():
    if material_path.is_dir() and fun_获取数字(material_path.stem) >= start_stem:
        all_path.append(material_path)
all_path.sort(key=lambda k: fun_获取数字(k.stem))

# 遍历文件夹开始上传素材到淘宝
for material_path in all_path:
    # 打开上传页面
    driver.get(
        "https://item.upload.taobao.com/sell/v2/publish.htm?catId=201160807&smartRouter=true&keyProps=%7B%7D&newRouter=1&paramCacheId=merge_router_cache_389353239_1694673279246_988&x-gpf-submit-trace-id=213e259e16946732792022149e0968"
    )

    # 标题
    ActionChains(driver).send_keys_to_element(
        driver.find_element(
            By.CSS_SELECTOR,
            "#sell-field-title > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > span > span > span > span > input",
        ),
        material_path.stem,
    ).perform()

    # 发货时间
    fun_js点击(
        selector=r"#sell-field-deliveryTimeType > div.sell-component-info-wrapper-component-wrap > div > div > span > span:nth-child(1) > label",
        driver=driver,
    )

    # 价格
    ActionChains(driver).send_keys_to_element(
        driver.find_element(
            By.CSS_SELECTOR,
            "#sell-field-price > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
        ),
        "3.8",
    ).perform()

    # 库存
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-quantity > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
    )
    ele.send_keys(Keys.CONTROL, "a")
    time.sleep(0.5)
    ele.send_keys(Keys.DELETE)
    time.sleep(0.5)
    ele.send_keys("888888")

    # 商家编码
    driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-outerId > div.sell-component-info-wrapper-component-wrap > div > div > span > span > span > span > input",
    ).send_keys(material_path.stem)

    # 翻页
    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.PAGE_DOWN)

    # 点击上传首图的图标
    driver.find_element(
        By.CSS_SELECTOR,
        "#struct-mainImagesGroup > div > div:nth-child(1) > div > div > div",
    ).click()

    # 切换到上传图片iframe
    ele = driver.find_element(By.CSS_SELECTOR, "#mainImagesGroup")
    driver.switch_to.frame(ele)

    # 点击ST文件夹
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "#sucai-tu-selector-container > div.Home_body__P3Ncj.Home_withoutHeaderBody__DxGAT > div.PicGroupList_aside__fp5JR > div > div > div > div > ul > li:nth-child(3) > div",
    )
    ele.click()

    # 点击首图图片
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "#sucai_tu_selector_scrollMain > div > div > label > div > img",
    )
    ele.click()

    # 切换到主框架
    # 并且下翻页
    driver.switch_to.default_content()
    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.PAGE_DOWN)

    # 详情PC端返回到旧版本
    ele = driver.find_element(
        by=By.CSS_SELECTOR,
        value=r"#sell-field-descRepublicOfSell > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div.liteDetailEditorContainer--Oq8UB > div.distinguish--YMXaf > div:nth-child(2) > button",
    )
    ele.click()

    # 确定
    ele = driver.find_element(
        By.CSS_SELECTOR,
        r"body > div.ln-overlay-wrapper.opened > div.ln-dialog.ln-closeable.ln-overlay-inner.ln-dialog-quick > div.ln-dialog-footer.ln-align-right > button.ln-btn.ln-medium.ln-btn-primary.ln-dialog-btn",
    )
    ele.click()

    # 使用本文编辑
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-descType > div.sell-component-info-wrapper-component-wrap > div > div > span > span:nth-child(1) > label",
    )
    ele.click()

    # 上传图片按钮
    ele = driver.find_element(By.CSS_SELECTOR, "#cke_56")
    ele.click()

    # 切换到图片上传框
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "body > div.next-overlay-wrapper.opened > div.next-overlay-inner.media-dialog.ck-dialog > div > iframe",
    )
    driver.switch_to.frame(ele)

    # 搜索框输入商家编码
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "#container > div > div.hasEms.container > div.main > div > div.opt-body > div.list > div.search-bar > div:nth-child(3) > span.next-input.next-input-single.next-input-medium.clear > input[type=text]",
    )
    ele.send_keys(f"{material_path.stem}-xq_")
    time.sleep(2)

    # 回车确定搜索
    ele.send_keys(Keys.ENTER)
    time.sleep(2)

    # 构建图片列表
    eles = driver.find_elements(By.CSS_SELECTOR, "#items > div")
    eles.sort(key=lambda k: fun_获取ele数字(k))

    num = 1
    for ele in eles[:20]:
        if num == fun_获取ele数字(ele):
            ele.click()
            time.sleep(0.5)

            num += 1
        else:
            raise IndexError("图片不正确，缺少图片")

    # 点击确定
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "#container > div > div.hasEms.container > div.main > div > div.opt-footer > div.btn.btn-blue",
    )
    ele.click()

    # 如果图片大于20张图片
    if len(eles) > 20:
        # 切回到主框架
        driver.switch_to.default_content()

        # 上传图片按钮
        ele = driver.find_element(By.CSS_SELECTOR, "#cke_56")
        ele.click()

        # 切换到图片上传框
        ele = driver.find_element(
            By.CSS_SELECTOR,
            "body > div.next-overlay-wrapper.opened > div.next-overlay-inner.media-dialog.ck-dialog > div > iframe",
        )
        driver.switch_to.frame(ele)

        # 搜索框输入商家编码
        ele = driver.find_element(
            By.CSS_SELECTOR,
            "#container > div > div.hasEms.container > div.main > div > div.opt-body > div.list > div.search-bar > div:nth-child(3) > span.next-input.next-input-single.next-input-medium.clear > input[type=text]",
        )
        ele.send_keys(f"{material_path.stem}-xq_")
        time.sleep(2)

        # 回车确定搜索
        ele.send_keys(Keys.ENTER)
        time.sleep(2)

        # 构建图片列表
        eles = driver.find_elements(By.CSS_SELECTOR, "#items > div")
        eles.sort(key=lambda k: fun_获取ele数字(k))

        for ele in eles[20:40]:
            if num == fun_获取ele数字(ele):
                ele.click()
                time.sleep(0.5)

                num += 1
            else:
                raise IndexError("图片不正确，缺少图片")

        # 点击确定
        ele = driver.find_element(
            By.CSS_SELECTOR,
            "#container > div > div.hasEms.container > div.main > div > div.opt-footer > div.btn.btn-blue",
        )
        ele.click()

    # 切回到主框架
    driver.switch_to.default_content()

    # 手机端使用本文编辑
    ele = driver.find_element(
        By.CSS_SELECTOR,
        "#sell-field-wirelessDescType > div.sell-component-info-wrapper-component-wrap > div > div > span > span:nth-child(1) > label",
    )
    ele.click()

    # 切回主框架
    # 往下翻页
    driver.switch_to.default_content()
    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.PAGE_DOWN)

    # 放入仓库
    fun_js点击(
        driver=driver,
        selector="#sell-field-startTime > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > span > span:nth-child(3) > label",
    )
    time.sleep(2)

    # 提交产品
    fun_js点击(driver=driver, selector="#struct-buttons > button")
    time.sleep(3)

    success_ele = driver.find_element(
        By.CSS_SELECTOR,
        "#success-container > div > div > p",
    )
    wait = WebDriverWait(driver=driver, timeout=10)
    wait.until(lambda d: success_ele.is_displayed())